<a href="https://colab.research.google.com/github/Nevesm5/Datario/blob/main/analise_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install basedosdados

In [ ]:
import basedosdados as bd
import pandas as pd
import numpy as np

In [ ]:
bairro = bd.read_sql("SELECT * FROM datario.dados_mestres.bairro", billing_project_id= "746089026690", reauth=True)


Downloading: 100%|██████████| 164/164 [00:02<00:00, 64.56rows/s]


Tabelas (1 a 5) \
datario.dados_mestres.bairro \
datario.adm_central_atendimento_1746.chamado

1-Quantos chamados foram abertos no dia 01/04/2023?

In [ ]:
chamado = bd.read_sql("SELECT * FROM datario.adm_central_atendimento_1746.chamado where date(data_inicio)='2022-04-01'", billing_project_id= "746089026690")

Downloading: 100%|██████████| 2251/2251 [00:00<00:00, 2466.79rows/s]


In [ ]:
chamado.shape[0]

2251

2-Qual o tipo de chamado que teve mais teve chamados abertos no dia 01/04/2023?

In [ ]:
chamado.tipo.value_counts().reset_index().rename(columns={'index':'tipo', 'tipo':'quantidade'}).head(1)

,tipo,quantidade
0,Iluminação Pública,332


3-Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

In [ ]:
np.sort(chamado.columns)

array(['categoria', 'data_alvo_diagnostico', 'data_alvo_finalizacao',
       'data_fim', 'data_inicio', 'data_particao',
       'data_real_diagnostico', 'dentro_prazo', 'id_bairro', 'id_chamado',
       'id_logradouro', 'id_origem_ocorrencia', 'id_subtipo',
       'id_territorialidade', 'id_tipo', 'id_unidade_organizacional',
       'id_unidade_organizacional_mae', 'justificativa_status',
       'latitude', 'longitude', 'nome_unidade_organizacional',
       'numero_logradouro', 'prazo_tipo', 'prazo_unidade', 'reclamacoes',
       'situacao', 'status', 'subtipo', 'tempo_prazo', 'tipo',
       'tipo_situacao', 'unidade_organizacional_ouvidoria'], dtype=object)

In [ ]:
bairro.columns

Index(['id_bairro', 'nome', 'id_area_planejamento', 'id_regiao_planejamento',
       'nome_regiao_planejamento', 'id_regiao_administrativa',
       'nome_regiao_administrativa', 'subprefeitura', 'area', 'perimetro',
       'geometry_wkt', 'geometry'],
      dtype='object')

In [ ]:
chamado.merge(bairro, on='id_bairro')['nome'].value_counts().head(3)

,nome
Campo Grande,140
Tijuca,88
Botafogo,74


In [ ]:
chamado.merge(bairro, on='id_bairro').groupby('nome').size().sort_values(ascending=False).head(3)

,0
nome,
Campo Grande,140
Tijuca,88
Botafogo,74


4 - Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [ ]:
bairro.columns

Index(['id_bairro', 'nome', 'id_area_planejamento', 'id_regiao_planejamento',
       'nome_regiao_planejamento', 'id_regiao_administrativa',
       'nome_regiao_administrativa', 'subprefeitura', 'area', 'perimetro',
       'geometry_wkt', 'geometry'],
      dtype='object')

In [ ]:
chamado.merge(bairro, on='id_bairro')['subprefeitura'].value_counts().head(1)

,subprefeitura
Zona Norte,602


5 - Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

In [ ]:
chamado_bairro = chamado.merge(bairro, on='id_bairro')

In [ ]:
chamado_bairro.columns

Index(['id_chamado', 'id_origem_ocorrencia', 'data_inicio', 'data_fim',
       'id_bairro', 'id_territorialidade', 'id_logradouro',
       'numero_logradouro', 'id_unidade_organizacional',
       'nome_unidade_organizacional', 'id_unidade_organizacional_mae',
       'unidade_organizacional_ouvidoria', 'categoria', 'id_tipo', 'tipo',
       'id_subtipo', 'subtipo', 'status', 'longitude', 'latitude',
       'data_alvo_finalizacao', 'data_alvo_diagnostico',
       'data_real_diagnostico', 'tempo_prazo', 'prazo_unidade', 'prazo_tipo',
       'dentro_prazo', 'situacao', 'tipo_situacao', 'justificativa_status',
       'reclamacoes', 'data_particao', 'nome', 'id_area_planejamento',
       'id_regiao_planejamento', 'nome_regiao_planejamento',
       'id_regiao_administrativa', 'nome_regiao_administrativa',
       'subprefeitura', 'area', 'perimetro', 'geometry_wkt', 'geometry'],
      dtype='object')

In [ ]:
for i in chamado_bairro.columns:
  if  (chamado_bairro[i].isna().sum()) > 1:
    print (i)

data_fim
numero_logradouro
longitude
latitude
data_alvo_finalizacao
data_alvo_diagnostico
data_real_diagnostico
tempo_prazo
prazo_unidade
prazo_tipo
justificativa_status


Nova base chamados e eventos

In [ ]:
chamado = bd.read_sql("SELECT * FROM datario.adm_central_atendimento_1746.chamado \
where date(data_inicio) between '2022-01-01' and '2023-12-31'  \
", billing_project_id= "746089026690")

Downloading: 100%|██████████| 1634336/1634336 [07:58<00:00, 3415.50rows/s]


In [ ]:
eventos = bd.read_sql("SELECT * FROM datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos", billing_project_id= "746089026690")

Downloading: 100%|██████████| 4/4 [00:00<00:00, 25.56rows/s]


In [ ]:
chamado.head(2)

,id_chamado,id_origem_ocorrencia,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,...,data_real_diagnostico,tempo_prazo,prazo_unidade,prazo_tipo,dentro_prazo,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao
0,18622711,11,2023-04-28 05:48:08,2023-04-29 15:42:21,5,1,61812,175,8,RIOLUZ - Companhia Municipal de Energia e Ilum...,...,NaT,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2023-04-01
1,18585454,11,2023-04-18 19:08:42,2023-04-21 01:40:52,5,1,61937,5,8,RIOLUZ - Companhia Municipal de Energia e Ilum...,...,NaT,<NA>,D,F,No prazo,Encerrado,Atendido,None,0,2023-04-01


In [ ]:
eventos

,ano,data_inicial,data_final,evento,taxa_ocupacao
0,18/02 a 21/02 de 2023,2023-02-18,2023-02-21,Carnaval,0.9554
1,30-31/12 e 01/01 (2022-2023),2022-12-30,2023-01-01,Reveillon,0.9251
2,08/09 a 11/09 de 2022,2022-09-08,2022-09-11,Rock in Rio,0.9451
3,02/09 a 04/09 de 2022,2022-09-02,2022-09-04,Rock in Rio,0.8184


In [ ]:
query = """SELECT
  *
FROM
  datario.adm_central_atendimento_1746.chamado AS A
JOIN
  datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos AS B
ON
   A.data_inicio = B.data_inicial """

In [ ]:
bd.read_sql(query,  billing_project_id= "746089026690")






Downloading: 0rows [00:00, ?rows/s]


,id_chamado,id_origem_ocorrencia,data_inicio,data_fim,id_bairro,id_territorialidade,id_logradouro,numero_logradouro,id_unidade_organizacional,nome_unidade_organizacional,...,situacao,tipo_situacao,justificativa_status,reclamacoes,data_particao,ano,data_inicial,data_final,evento,taxa_ocupacao


6-Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [ ]:
chamado[chamado['subtipo']=='Perturbação do sossego'].shape[0]

42830

7-Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [ ]:
pertubacao = chamado[chamado['subtipo']=='Perturbação do sossego']

In [ ]:
eventos.data_inicial = eventos.data_inicial.astype(str)

In [ ]:
pertubacao['data_inicial'] =  pertubacao['data_inicio'].astype(str).apply(lambda x : x[0:10])

In [ ]:
pert_evento = pertubacao.merge(eventos, on='data_inicial')

In [ ]:
pert_evento['evento'].unique()


array(['Carnaval', 'Rock in Rio', 'Reveillon'], dtype=object)

In [ ]:
pert_evento['id_chamado']


,id_chamado
0,18328955
1,18327681
2,18329445
3,18328957
4,18328714
...,...
290,18075894
291,18077088
292,18077935
293,18076842


8-Quantos chamados desse subtipo foram abertos em cada evento?

In [ ]:
pert_evento['evento'].value_counts()

,evento
Rock in Rio,177
Carnaval,74
Reveillon,44


9-Qual evento teve a maior média diária de chamados abertos desse subtipo?

In [ ]:
pert_evento.groupby(['evento','data_inicial']).sum()

numero_logradouro    longitude     latitude  \
evento      data_inicial                                                
Carnaval    2023-02-18               129799 -2641.326537 -1399.098031   
Reveillon   2022-12-30                12451 -1557.655605  -825.165252   
Rock in Rio 2022-09-02                77294 -3722.849353 -1971.864654   
            2022-09-08                26631 -2814.198043 -1490.210793   

                          tempo_prazo  reclamacoes  taxa_ocupacao  
evento      data_inicial                                           
Carnaval    2023-02-18              0            4        70.6996  
Reveillon   2022-12-30              0            5        40.7044  
Rock in Rio 2022-09-02              0           15        81.8400  
            2022-09-08              0            9        72.7727

In [ ]:
pert_evento[['id_chamado', 'evento', 'data_inicial']].groupby(['evento', 'data_inicial']).count().groupby('evento').mean().sort_values(by='id_chamado', ascending=False)

,id_chamado
evento,
Rock in Rio,88.5
Carnaval,74.0
Reveillon,44.0


10-Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.

In [ ]:

media_diaria_eventos = pert_evento[['id_chamado', 'evento', 'data_inicial']].groupby(['evento', 'data_inicial']).count().groupby('evento').mean().sort_values(by='id_chamado', ascending=False)
media_diaria_total = pertubacao.shape[0] / ((pd.to_datetime('2023-12-31') - pd.to_datetime('2022-01-01')).days + 1)
print("Média diária de chamados 'Perturbação do sossego' durante os eventos:")
print(media_diaria_eventos)
print("\nMédia diária de chamados 'Perturbação do sossego' no período total:")
"imprimindo a media_diaria_total com 2 casas decimais"

print(f"{media_diaria_total:.2f}")


Média diária de chamados 'Perturbação do sossego' durante os eventos:
             id_chamado
evento                 
Rock in Rio        88.5
Carnaval           74.0
Reveillon          44.0

Média diária de chamados 'Perturbação do sossego' no período total:
58.67
